<a href="https://colab.research.google.com/github/WanjohiWanjohi/Term_deposits_uptake_predictor/blob/master/bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier

#drive.mount('/content/drive')

In [ ]:
#get the file 
#df =  pd.read_csv('bank-additional/bank-additional/bank-additional-full.csv' , sep=";" , engine='python')
df = pd.read_csv('/content/drive/My Drive/bank-additional-full.csv' , sep=";")
df.to_pickle("./bankcsv.pkl")
df

 The duration skews the analysis as it is not known before a call is made and should therefore be dropped before we feed this data to our model 

In [ ]:
df.drop('duration' , axis=1 , inplace=True)
df.info() , df.describe()


From the information , we can see we have no null entries in the dataframe 
Hence , we can go forward and process our data by:
1. Encoding Categorical Data
2. Scale
3. Class Balance in target variable
5. Aggregating and Transformation
6. Dimension reduction





In [215]:
def convert_categorical(dataframe , columns):
   df = dataframe[columns]
   converted_df = pd.get_dummies(df)
   res = pd.concat([df, converted_df], axis=1)
   return res

def get_column_counts(df ):
  """
  Get count of unique items in the dataframe column
  """
  for c in df.columns:
    print(c)
    print(df[c].value_counts())

def scale_columns(df , column):
  """"
  Scale columns using MinMax scaler
  """
  col = df[column]
  scaler = MinMaxScaler() 
  num2 = scaler.fit_transform(col)
  num2 = pd.DataFrame(num2, columns = col.columns)
  return num2

def upsample(df , majority_col, minority_col):
  """"
  Perform upsampling by resampling while replacing
  """
  # we seperate the classes
  majority_class = df[majority_col]
  minority_class = df[minority_col]
  #Upsample minority class
  df_minority_upsampled = resample(minority_class, 
                                 replace=True,     # sample with replacement
                                 n_samples=36548,    # to match majority class
                                 random_state=123) # reproducible results
  # Combine majority class with upsampled minority class
  df_upsampled = pd.concat([majority_class, df_minority_upsampled])
  return df_upsampled

def convert_dates(df , column , format):
  """
  convert date and months to numerical format

  """
  months = []
  day_of_week = []
  #convert column items to strings
  
    #format months
  if format == "%b":
    for index, row in df[column].items():
      datetime_object = datetime.datetime.strptime(row , "%b")
      month_number = datetime_object.month
      months.append(month_number)
    return months
    ## format days
  elif format == "%a":
    for index, row in df[column].items():
      datetime_object = datetime.datetime.strptime(row , "%a")
      day_number = datetime_object.weekday()
      day_of_week.append(day_number)
    return day_of_week
def get_correlation_map(correlation_matrix):
  """
  return a heatmap of the correlation
  """
  plt.figure(figsize=(20,5))
  return sns.heatmap(correlation_matrix)


def get_target_sample(df , column , sample_number):
  """
  """
  sample = df[column].sample(n=int(sample_number), random_state=1)
  return sample

def pca_reduction(df , components):
  """
  """
  feat_cols = df.columns.tolist()
  pca = PCA(n_components=components).fit(df)
  var_exp = pca.explained_variance_ratio_
  #apply the dimensionality reduction
  transformed = pca.transform(df)
  
  return transformed

def logistic_reg(X,y):
  """
  """
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.1,random_state=1)
  lr = LogisticRegressionCV(random_state=0).fit(X_train, y_train)
  prediction=lr.predict(X_test)
  return prediction , y_train

def get_roc_auc_score(true_y , predicted_y , avg = "macro"):
  """"
  """
  score = roc_auc_score(true_y , predicted_y ,average=avg )
  return score

def XGBoost(X , y):
  """
  """
  # convert the dataset into a Dmatrix that gives it  performance and efficiency gains.
  data_dmatrix = xgb.DMatrix(data=X,label=y)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
  xg_reg = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
  xg_reg.fit(X_train,y_train)
  preds = xg_reg.predict(X_test)
  params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
  cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="auc", as_pandas=True, seed=123)
  return preds , cv_results

def multi_layer_percep(X , y):
  """
  
  """
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
  clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
  predict = clf.predict(X_test)
  return y_train , predict


#### Encode categorical data 
We use the get_dummies() function availed by pandas

0 indicates non existent while 1 indicates existent.


In [7]:
categorical = ['job' , 'marital' , 'education' , 'contact' , 'housing' , 'default' , 'loan' , 'poutcome']
target = ['y' ]
binned = ['pdays']
dates = ['month' , 'day_of_week']

In [ ]:
df = pd.read_pickle("./bankcsv.pkl")
converted_df = convert_categorical(df , categorical)
get_column_counts(converted_df)

In [9]:
month = convert_dates(df , dates[0]  , '%b')
day = convert_dates(df , dates[1] , '%a')


#### Scale
We scale the pdays , employee numbers and age columns column to have a restricted range

In [16]:
scaled = scale_columns(df , binned)
scaled_emp_no = scale_columns(df , ['nr.employed'])
scaled_age = scale_columns(df , ['age'])

#### Class Balancing
Imbalanced classes put “accuracy” out of business. This is a surprisingly common problem in machine learning (specifically in classification), occurring in datasets with a disproportionate ratio of observations in each class as is the case in our dataset with the target variable


In [17]:
#check existence of class imbalance
get_column_counts(df[target])

y
no     36548
yes     4640
Name: y, dtype: int64


With a population of 41,188 , the yes population makes up only 11% of the population
We handle this by upsampling the minority class(the yes)

1. First, we'll separate observations from each class into different DataFrames.
2. Next, we'll resample the minority class with replacement, setting the number of samples to match that of the majority class.
3. Finally, we'll combine the up-sampled minority class DataFrame with the original majority class DataFrame.



In [55]:
#   we start by encoding the content of the target column
target_df = convert_categorical(df , target)

#   use upsample function
upsampled = upsample(target_df ,'y_no' , 'y_yes' )

target_column = pd.DataFrame(upsampled , columns=["deposit"])

#### Transform and Aggregate
*First , let us bring it all together into one dataframe*


In [ ]:
#combine the encoded , scaled and datetime formatted columns
## drop the categorical columns
converted_df.drop(categorical , axis=1 , inplace=True)
converted_df

# bring in the campaign columns and the age column
converted_df['age'] = scaled_age
converted_df['campaign'] = df['campaign']
converted_df['previous'] = df['previous']

#bring in the derived month column
converted_df['month'] = month

#bring in the scaled employee numbers and number of days that pass before a customer is contacted
converted_df['pdays'] = scaled
converted_df['employee_no'] = scaled_emp_no


converted_df

In [212]:
#calculate total number of contacts made with user
sum_column = converted_df['previous'] + converted_df['campaign']
converted_df['total_contacts'] = sum_column

#### Reduce the dimensions
45 columns is a large number ; additionally some of them report the same result
Dimensionality reduction can be done in two different ways:

  - By only keeping the most relevant variables from the original dataset (this technique is called feature selection)
  - By finding a smaller set of new variables, each being a combination of the input variables, containing basically the same information as the input variables (this technique is called dimensionality reduction)


###### Low Variance filtering


In [ ]:
variance = converted_df.var()
variance


We can safely drop the day column since it varies never


In [ ]:
converted_df.drop('day' , axis=1 , inplace=True)
converted_df.astype('float64').dtypes

#### Principal Compnent Analysis




In [ ]:
transformed_features = pca_reduction(converted_df , 10)


### Modeling
We use three main model approaches :
  - Logistic regression
  - XGBoost
  - Multi Layer Perceptron
  

#### Logistic Regression

In [155]:
X = transformed_features  
y = get_target_sample(target_column , 'deposit' , 41188  )

In [ ]:
pred, true = logistic_reg(X , y)
#get evaluation of mode

#save model

#### XGBoost

In [210]:
pred , cv_results = XGBoost(X, y)
#get evaluation of model
cv_results
#save model

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.523309,0.000815,0.499027,0.00436


#### Multi Layer Perceptron

In [216]:
multi_layer_percep(X , y)


(1928     0
 31617    0
 9584     1
 12035    1
 34658    1
         ..
 30865    0
 1327     1
 10278    0
 35859    1
 10469    1
 Name: deposit, Length: 30891, dtype: uint8,
 array([1, 1, 1, ..., 1, 0, 0], dtype=uint8))

# References
http://deeplearning.net/tutorial/mlp.html